In [2]:
library(dplyr)   
library(caret)
library(tidyverse)
library(recipes)
library(pROC)
library(caTools)
library(UBL)

set.seed(1)
options(repr.plot.width = 20, repr.plot.height =10, repr.plot.res = 600)



load(file = "C:/Users/abonini/Desktop/Analysis/Sources/LIVERTRANSPLANT/Modello/df_finale.Rdata")

df <- df_finale

## Data partition

In [3]:
#make this example reproducible
set.seed(1)

#use 70% of dataset as training set and 30% as test set
sample <- sample.split(df$deg_cat, SplitRatio = 0.7)
train  <- subset(df, sample == TRUE)
test   <- subset(df, sample == FALSE)


TestClasses <- test$deg_cat
TestData <- subset(test,select= -deg_cat)

In [4]:
train <- SmoteClassif(deg_cat ~ ., train, k = 27, dist = "HEOM")

#view distribution of response variable in new dataset
table(train$deg_cat)

TrainClasses <- train$deg_cat
TrainData <- subset(train,select= -deg_cat)


   fast NotFast 
    488     488 

## Models

In [5]:
ctrl <- trainControl(method = "repeatedcv", 
                 number = 10, 
                 savePredictions = "final",classProbs=TRUE,verboseIter=TRUE, search= "random") 

#summaryFunction = multiClassSummary (to keep all metrics)

In [6]:
model_rf <- train(x=TrainData, y=TrainClasses, 
               method = "ranger",
               trControl = ctrl) 

+ Fold01.Rep1: min.node.size=14, mtry=53, splitrule=gini 
- Fold01.Rep1: min.node.size=14, mtry=53, splitrule=gini 
+ Fold01.Rep1: min.node.size= 5, mtry=14, splitrule=extratrees 
- Fold01.Rep1: min.node.size= 5, mtry=14, splitrule=extratrees 
+ Fold01.Rep1: min.node.size= 8, mtry=39, splitrule=gini 
- Fold01.Rep1: min.node.size= 8, mtry=39, splitrule=gini 
+ Fold02.Rep1: min.node.size=14, mtry=53, splitrule=gini 
- Fold02.Rep1: min.node.size=14, mtry=53, splitrule=gini 
+ Fold02.Rep1: min.node.size= 5, mtry=14, splitrule=extratrees 
- Fold02.Rep1: min.node.size= 5, mtry=14, splitrule=extratrees 
+ Fold02.Rep1: min.node.size= 8, mtry=39, splitrule=gini 
- Fold02.Rep1: min.node.size= 8, mtry=39, splitrule=gini 
+ Fold03.Rep1: min.node.size=14, mtry=53, splitrule=gini 
- Fold03.Rep1: min.node.size=14, mtry=53, splitrule=gini 
+ Fold03.Rep1: min.node.size= 5, mtry=14, splitrule=extratrees 
- Fold03.Rep1: min.node.size= 5, mtry=14, splitrule=extratrees 
+ Fold03.Rep1: min.node.size= 8, mtr

In [7]:
model_rrf <- train(x=TrainData, y=TrainClasses, 
               method = "RRF",
               trControl = ctrl) 

Registered S3 method overwritten by 'RRF':
  method      from        
  plot.margin randomForest



+ Fold01.Rep1: mtry=44, coefReg=0.40539, coefImp=0.3840 
- Fold01.Rep1: mtry=44, coefReg=0.40539, coefImp=0.3840 
+ Fold01.Rep1: mtry=19, coefReg=0.87669, coefImp=0.5726 
- Fold01.Rep1: mtry=19, coefReg=0.87669, coefImp=0.5726 
+ Fold01.Rep1: mtry=44, coefReg=0.03929, coefImp=0.5922 
- Fold01.Rep1: mtry=44, coefReg=0.03929, coefImp=0.5922 
+ Fold02.Rep1: mtry=44, coefReg=0.40539, coefImp=0.3840 
- Fold02.Rep1: mtry=44, coefReg=0.40539, coefImp=0.3840 
+ Fold02.Rep1: mtry=19, coefReg=0.87669, coefImp=0.5726 
- Fold02.Rep1: mtry=19, coefReg=0.87669, coefImp=0.5726 
+ Fold02.Rep1: mtry=44, coefReg=0.03929, coefImp=0.5922 
- Fold02.Rep1: mtry=44, coefReg=0.03929, coefImp=0.5922 
+ Fold03.Rep1: mtry=44, coefReg=0.40539, coefImp=0.3840 
- Fold03.Rep1: mtry=44, coefReg=0.40539, coefImp=0.3840 
+ Fold03.Rep1: mtry=19, coefReg=0.87669, coefImp=0.5726 
- Fold03.Rep1: mtry=19, coefReg=0.87669, coefImp=0.5726 
+ Fold03.Rep1: mtry=44, coefReg=0.03929, coefImp=0.5922 
- Fold03.Rep1: mtry=44, coefReg

## Results

In [8]:
print(model_rf)
plot(model_rf)

Random Forest 

976 samples
 73 predictor
  2 classes: 'fast', 'NotFast' 

No pre-processing
Resampling: Cross-Validated (10 fold, repeated 1 times) 
Summary of sample sizes: 878, 879, 879, 879, 878, 878, ... 
Resampling results across tuning parameters:

  min.node.size  mtry  splitrule   Accuracy   Kappa    
   5             14    extratrees  0.7940143  0.5878740
   8             39    gini        0.7725437  0.5449372
  14             53    gini        0.7715127  0.5428502

Accuracy was used to select the optimal model using the largest value.
The final values used for the model were mtry = 14, splitrule = extratrees
 and min.node.size = 5.


In [9]:
print(model_rrf)
plot(model_rrf) 

Regularized Random Forest 

976 samples
 73 predictor
  2 classes: 'fast', 'NotFast' 

No pre-processing
Resampling: Cross-Validated (10 fold, repeated 1 times) 
Summary of sample sizes: 879, 879, 878, 878, 878, 879, ... 
Resampling results across tuning parameters:

  mtry  coefReg     coefImp    Accuracy   Kappa    
  19    0.87668624  0.5725642  0.7408058  0.4815269
  44    0.03928998  0.5921755  0.6925836  0.3850784
  44    0.40539039  0.3840113  0.6792973  0.3585691

Accuracy was used to select the optimal model using the largest value.
The final values used for the model were mtry = 19, coefReg = 0.8766862
 and coefImp = 0.5725642.


In [10]:
# collect resamples
results <- resamples(list(ranger=model_rf,rrf=model_rrf))
# summarize the distributions
summary(results)
# boxplots of results
bwplot(results)
# dot plots of results
dotplot(results)


Call:
summary.resamples(object = results)

Models: ranger, rrf 
Number of resamples: 10 

Accuracy 
            Min.   1st Qu.    Median      Mean   3rd Qu.      Max. NA's
ranger 0.7448980 0.7634915 0.7795603 0.7940143 0.8200873 0.8673469    0
rrf    0.6836735 0.7244898 0.7295918 0.7408058 0.7525773 0.8265306    0

Kappa 
            Min.   1st Qu.    Median      Mean   3rd Qu.      Max. NA's
ranger 0.4897959 0.5261114 0.5590109 0.5878740 0.6400908 0.7346939    0
rrf    0.3673469 0.4489796 0.4591837 0.4815269 0.5053121 0.6530612    0


## Prediction

In [11]:
pred_rf <- predict(model_rf,TestData)  
pred_rf.prob <- predict(model_rf,TestData, type="prob")


result <- confusionMatrix(TestClasses,  pred_rf) 
result
result$byClass

Confusion Matrix and Statistics

          Reference
Prediction fast NotFast
   fast     102      44
   NotFast   63     210
                                       
               Accuracy : 0.7446       
                 95% CI : (0.7, 0.7857)
    No Information Rate : 0.6062       
    P-Value [Acc > NIR] : 1.685e-09    
                                       
                  Kappa : 0.4541       
                                       
 Mcnemar's Test P-Value : 0.08184      
                                       
            Sensitivity : 0.6182       
            Specificity : 0.8268       
         Pos Pred Value : 0.6986       
         Neg Pred Value : 0.7692       
             Prevalence : 0.3938       
         Detection Rate : 0.2434       
   Detection Prevalence : 0.3484       
      Balanced Accuracy : 0.7225       
                                       
       'Positive' Class : fast         
                                       

Sensitivity          Specificity       Pos Pred Value 
           0.6181818            0.8267717            0.6986301 
      Neg Pred Value            Precision               Recall 
           0.7692308            0.6986301            0.6181818 
                  F1           Prevalence       Detection Rate 
           0.6559486            0.3937947            0.2434368 
Detection Prevalence    Balanced Accuracy 
           0.3484487            0.7224767

In [12]:
roc_obj <- roc(TestClasses, as.numeric(pred_rf))
plot(roc_obj, main="Curva ROC")

Setting levels: control = fast, case = NotFast

Setting direction: controls < cases



In [13]:
roc_obj


Call:
roc.default(response = TestClasses, predictor = as.numeric(pred_rf))

Data: as.numeric(pred_rf) in 146 controls (TestClasses fast) < 273 cases (TestClasses NotFast).
Area under the curve: 0.7339

In [14]:
pred_rf.prob

fast,NotFast
0.80523333,0.1947667
0.08856667,0.9114333
0.20330000,0.7967000
0.56653333,0.4334667
0.43856667,0.5614333
0.22150000,0.7785000
0.47593333,0.5240667
0.39823333,0.6017667
0.16706667,0.8329333
0.24626667,0.7537333
